# Añadir una nueva llamada al sistema

obtener el kernel de linux:

In [ ]:
mkdir myproject2
cd myproject2
wget http://www.kernel.org/pub/linux/kernel/v3.0/linux-3.5.2.tar.bz2
tar -xjvf linux-3.5.2.tar.bz2
cd linux-3.5.2/

### se añade el archivo con la nueva primitiva

In [ ]:
cd kernel
wd=`pwd`
echo "#include <linux/linkage.h>
#include <linux/kernel.h>
asmlinkage int sys_secuencia_syscall(int x) {
    return (x/2.24);
}" >> ${wd}/secuencia_syscall.c

Este comando hace genera automaticamente la llamada al sistema ```syscall.c```

### Modificación del Makefile de linux

In [ ]:
sed -i 's/async.o range.o groups.o lglock.o/async.o range.o groups.o lglock.o secuencia_syscall.o/g' "Makefile"

### Añadiendo nuestra nueva llamada a la tabla de llamadas al sistema

In [ ]:
cd ..
cd arch/x86/syscalls/
ls

* En esta carpeta vemos 2 archivos importantes ```syscall_32.tbl``` y ```syscall_64.tbl```. En estos archivos se definen los códigos de llamadas al sistema para cada arquitectura (32 y 64 bits respectivamente)


editaremos ```syscall_64.tbl``` :

In [ ]:
gedit syscall_64.tbl

debemos añadir la lı́nea correspondiente a nuestra llamada a sistema, junto con su
numero, nombre de la llamada y la función del kernel que la implementa:

```
310	64	process_vm_readv	sys_process_vm_readv
311	64	process_vm_writev	sys_process_vm_writev
312	common	kcmp			sys_kcmp```

linea a añadir: ```313	common	secuencia_syscall	    sys_secuencia_syscall```

## Compilar la imagen de linux

In [ ]:
cd ../../..
su
make oldconfig

```make oldconfig``` con esto, copiamos las configuraciones de compilación del Kernel que estamos ejecutando,
para que se compilen en el Kernel modificado, y finalmente ```make``` para compilar el kernel

In [ ]:
make
make modules
make modules_install 
make install

### Instalando la imagen de linux y configurando el grub

Una vez terminado, la consola dirá en donde dejó la imagen:

```
Setup is 16848 bytes (padded to 16896 bytes).
System is 4649 kB
CRC 76a8a8f3
Kernel: arch/x86/boot/bzImage is ready (#5)
linux-3.5.2 $ ```

para instalarla, debemos copiar bzImage desde donde está, hasta ```/boot:```

In [ ]:
cp arch/x86/boot/bzImage /boot